In [76]:
# Setting seed to regenerate the same results
from __future__ import print_function
import tensorflow as tf
import numpy as np
import random
import os

SEED_VALUE = 0
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [77]:
import glob
from keras.preprocessing import image as kImage

def fetch_data(dataset_path, target_size=(240, 320)):
    X_img = sorted(glob.glob(os.path.join(dataset_path, 'x','*.png')))
    Y_truth = sorted(glob.glob(os.path.join(dataset_path, 'y','*.png')))

    N = len(X_img)
    print(f"Fetching {N} images from {dataset_path}/x")

    X = []
    Y = []

    for i in range(N):
        # Load input image
        x = kImage.load_img(X_img[i], target_size=target_size)
        x = kImage.img_to_array(x)
        X.append(x)
        
        # Load ground-truth label and encode it to label 0 and 1
        x = kImage.load_img(Y_truth[i], target_size=target_size, color_mode = "grayscale")
        x = kImage.img_to_array(x)
        x = np.floor(x/255.0)
        Y.append(x)

    X = np.asarray(X)
    Y = np.asarray(Y)

    # Shuffle the data
    shuffled_indexes = list(range(N))
    np.random.shuffle(shuffled_indexes)
    X = X[shuffled_indexes]
    Y = Y[shuffled_indexes]
    
    return X, Y

In [78]:
import keras
from keras.models import Model
from keras.layers import Conv2DTranspose, Input

def initModel():
    ### Encoder
    net_input = Input(shape=(240,320,3))
    vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet.h5', input_tensor=net_input)

    # Make first 17 layers non-trainable
    for layer in vgg16.layers[:17]:
      layer.trainable = False
    
    x = vgg16.layers[-2].output # 2nd layer from the last, block5_conv3
    
    ### Decoder
    x = Conv2DTranspose(256, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Conv2DTranspose(128, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Conv2DTranspose(32, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Conv2DTranspose(1, (1,1), activation='sigmoid', padding='same')(x)
    
    model = Model(inputs=vgg16.input, outputs=x)

    model.compile(loss=keras.losses.binary_crossentropy, optimizer=tf.keras.optimizers.legacy.RMSprop(learning_rate=5e-4), metrics=['accuracy'])

    return model

In [79]:
# load data
dataset_path = "./dataset/train"
X, Y = fetch_data(dataset_path)

#init the model
model = initModel()

early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10)
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

model.fit(X, Y, batch_size=1, epochs=100, verbose=2, validation_split=0.2, callbacks=[reduce, early], shuffle=True)
model.save('./weights/trained_weights.h5')

Fetching 23 images from ./dataset/train/x
Epoch 1/100
18/18 - 17s - loss: 0.0998 - accuracy: 0.9612 - val_loss: 0.0478 - val_accuracy: 0.9766 - lr: 5.0000e-04 - 17s/epoch - 955ms/step
Epoch 2/100
18/18 - 17s - loss: 0.0359 - accuracy: 0.9767 - val_loss: 0.0357 - val_accuracy: 0.9771 - lr: 5.0000e-04 - 17s/epoch - 940ms/step
Epoch 3/100
18/18 - 17s - loss: 0.0329 - accuracy: 0.9795 - val_loss: 0.0423 - val_accuracy: 0.9829 - lr: 5.0000e-04 - 17s/epoch - 945ms/step
Epoch 4/100
18/18 - 17s - loss: 0.0331 - accuracy: 0.9860 - val_loss: 0.0305 - val_accuracy: 0.9863 - lr: 5.0000e-04 - 17s/epoch - 949ms/step
Epoch 5/100
18/18 - 17s - loss: 0.0252 - accuracy: 0.9888 - val_loss: 0.0371 - val_accuracy: 0.9855 - lr: 5.0000e-04 - 17s/epoch - 956ms/step
Epoch 6/100
18/18 - 17s - loss: 0.0238 - accuracy: 0.9891 - val_loss: 0.0384 - val_accuracy: 0.9879 - lr: 5.0000e-04 - 17s/epoch - 954ms/step
Epoch 7/100
18/18 - 17s - loss: 0.0203 - accuracy: 0.9911 - val_loss: 0.0283 - val_accuracy: 0.9898 - lr: 

/Users/clef/Desktop/UBC/ML/SegNet/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [80]:
# load test data
dataset_path = "./dataset/test"
X, Y = fetch_data(dataset_path)

# predict
pred = model.predict(X, verbose=1, batch_size=1)

print(f"Ran model on {X.shape[0]} images and got average accuracy of {np.mean(np.equal(Y, np.round(pred)))*100}%")

Fetching 13 images from ./dataset/test/x
13/13 [==============================] - 9s 677ms/step
Ran model on 13 images and got average accuracy of 98.44671474358975%
